<h1>4장 텍스트 분류</h1>
<i>표현 모델과 생성 모델을 사용해 텍스트 분류하기</i>

<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter04.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 4장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>

### [선택사항] - <img src="https://colab.google/static/images/icons/colab.png" width=100>에서 패키지 선택하기


이 노트북을 구글 코랩에서 실행한다면 다음 코드 셀을 실행하여 이 노트북에서 필요한 패키지를  설치하세요.

---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---


In [1]:
%%capture
!pip install datasets

In [1]:
!pip install -U datasets

# 영화 리뷰 데이터셋

In [2]:
from datasets import load_dataset

# 데이터를 로드합니다.
data = load_dataset("rotten_tomatoes")
data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8530 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1066 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1066
    })
})

하이퍼 파라미터를 튜닝하고 싶을때 검증 set을 사용할 수 있다는게 무슨 말인가..?

In [3]:
data["train"][0, -1]

{'text': ['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
  'things really get weird , though not particularly scary : the movie is all portent and no content .'],
 'label': [1, 0]}

# 표현 모델로 텍스트 분류하기

표현모델 = 작업에 특화된 모델, 임베딩모델? = BERT?

BERT는 기초모델 - 이를 튜닝하면 임베딩모델이 되기도하고 작업모델이 되기도한다?

4.3 모델 선택
즉 어떤 작업을 할것이냐가 명확해야 모델 선택에 폭이 줄어들 것.
모델 선택 기준 : 특화작업, 사례, 언어호환성, 내부구조?, 크기, 성능


https://huggingface.co/spaces/mteb/leaderboard > 성능지표 리더보드?

추론 속도가 중요하다는데 어떻게 확인할 수 있는가?

# 작업에 특화된 모델 사용하기

In [5]:
from transformers import pipeline

# 허깅 페이스 모델 경로
model_path = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"

# 파이프라인으로 모델을 로드합니다.
pipe = pipeline(
    model=model_path,
    tokenizer=model_path,
    return_all_scores=True,
    device="cuda:0"
)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [7]:
import numpy as np
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

# 추론을 실행합니다.
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "text")), total=len(data["test"])):
    negative_score = output[0]["score"]
    positive_score = output[1]["score"]
    assignment = np.argmax([negative_score, positive_score])
    y_pred.append(assignment)

100%|██████████| 1066/1066 [00:05<00:00, 206.49it/s]


In [9]:
from sklearn.metrics import classification_report

def evaluate_performance(y_true, y_pred):
    """분류 리포트를 만들어 출력합니다."""
    performance = classification_report(
        y_true, y_pred,
        target_names=["Negative Review", "Positive Review"]
    )
    print(performance)

In [10]:
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.89      0.90      0.90       533
Positive Review       0.90      0.89      0.90       533

       accuracy                           0.90      1066
      macro avg       0.90      0.90      0.90      1066
   weighted avg       0.90      0.90      0.90      1066



중요한건 f1 스코어!

distilbert/distilbert-base-uncased-finetuned-sst-2-english  > 영화리뷰데이터에서 훈련한 모델? > |
                 
https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english - 모델의 특성을 어떻게 알 수 있는가?
                 
                 precision    recall  f1-score   support

Negative Review       0.89      0.90      0.90       533
Positive Review       0.90      0.89      0.90       533

       accuracy                           0.90      1066
      macro avg       0.90      0.90      0.90      1066
   weighted avg       0.90      0.90      0.90      1066

# 임베딩을 활용하여 분류 작업 수행하기

사전훈련된 모델이 없는 경우
미세튜닝 자원 많이 든다.
범용 임베딩 모델을 활용할 수 있다.

## 지도 학습 분류

In [17]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 임베딩으로 변환합니다.
train_embeddings = model.encode(data["train"]["text"], show_progress_bar=True)
test_embeddings = model.encode(data["test"]["text"], show_progress_bar=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

TypeError: Wrong key type: '686' of type '<class 'numpy.int64'>'. Expected one of int, slice, range, str or Iterable.

In [ ]:
train_embeddings.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

# 훈련 세트의 임베딩으로 로지스틱 회귀 모델을 훈련합니다.
clf = LogisticRegression(random_state=42)
clf.fit(train_embeddings, data["train"]["label"])

In [ ]:
# 테스트 세트 임베딩에 대해 예측을 수행합니다.
y_pred = clf.predict(test_embeddings)
evaluate_performance(data["test"]["label"], y_pred)

**팁!**  

분류기를 사용하지 않는다면 어떻게 할 수 있을까요? 분류기 대신 클래스별 임베딩을 평균하고 코사인 유사도를 적용하여 문서와 가장 잘 맞는 클래스를 예측할 수 있습니다.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity

# 타깃 레이블에 대한 문서의 임베딩을 모두 평균하여 타깃 임베딩을 만듭니다.
df = pd.DataFrame(np.hstack([train_embeddings, np.array(data["train"]["label"]).reshape(-1, 1)]))
averaged_target_embeddings = df.groupby(768).mean().values

# 테스트 임베딩과 가장 가까운 타깃 임베딩을 찾습니다.
sim_matrix = cosine_similarity(test_embeddings, averaged_target_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

# 모델을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)

## 데이터에 레이블이 없는 경우

In [ ]:
# 레이블의 임베딩을 만듭니다.
label_embeddings = model.encode(["A negative review",  "A positive review"])

레이블의 임베딩을 만든다?  
레이블을 정의하고 벡터화(수치화)한다고 이해하면 될까?

그 다음 문서와 레이블이 얼마나 유사한지 코사인 유사도를 활용해서 분류할 수 있다.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 각 문서와 가장 잘 맞는 레이블을 찾습니다.
sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

**팁!**  

설명을 다르게 하면 어떨까요? **"A very negative movie review"**와 **"A very positive movie review"**를 사용하면 어떻게 되는지 확인해 보세요!

In [ ]:
label_embeddings = model.encode(["A very negative movie review",
                                 "A very positive movie review"])

sim_matrix = cosine_similarity(test_embeddings, label_embeddings)
y_pred = np.argmax(sim_matrix, axis=1)

evaluate_performance(data["test"]["label"], y_pred)

# 생성 모델로 텍스트 분류하기

생성모델은 텍스트를 입력받아 텍스트를 생성하느 모델이다.
어떤 텍스트를 결과로 생성해야하는지 모르기때문에 원하는 답을 유도할 수 있도록 '프롬프트' 가 필요.


## T5 모델 사용하기

In [18]:
# 모델을 로드합니다.
pipe = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device="cuda:0"
)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [19]:
# 프롬프트를 추가합니다.
prompt = "Is the following sentence positive or negative? "
data = data.map(lambda example: {"t5": prompt + example['text']})
data

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 8530
    })
    validation: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
    test: Dataset({
        features: ['text', 'label', 't5'],
        num_rows: 1066
    })
})

In [20]:
# 추론을 실행합니다.
y_pred = []
for output in tqdm(pipe(KeyDataset(data["test"], "t5")), total=len(data["test"])):
    text = output[0]["generated_text"]
    y_pred.append(0 if text == "negative" else 1)

100%|██████████| 1066/1066 [00:58<00:00, 18.38it/s]


In [ ]:
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.83      0.85      0.84       533
Positive Review       0.85      0.83      0.84       533

       accuracy                           0.84      1066
      macro avg       0.84      0.84      0.84      1066
   weighted avg       0.84      0.84      0.84      1066



## ChatGPT로 분류하기

In [ ]:
# httpx 패키지의 proxies 매개변수 오류를 피하기 위해
# https://community.openai.com/t/error-with-openai-1-56-0-client-init-got-an-unexpected-keyword-argument-proxies/
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.0/457.0 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58

In [ ]:
import openai

# 클라이언트를 만듭니다.
client = openai.OpenAI(api_key="YOUR_KEY_HERE")

In [ ]:
def chatgpt_generation(prompt, document, model="gpt-4o-mini"):
    """프롬프트와 문서를 입력받아 출력을 생성합니다."""
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant."
            },
        {
            "role": "user",
            "content":   prompt.replace("[DOCUMENT]", document)
            }
    ]
    chat_completion = client.chat.completions.create(
      messages=messages,
      model=model,
      temperature=0
    )
    return chat_completion.choices[0].message.content

In [ ]:
# 프롬프트 템플릿을 정의합니다.
prompt = """Predict whether the following document is a positive or negative movie review:

[DOCUMENT]

If it is positive return 1 and if it is negative return 0. Do not give any other answers.
"""

# Predict the target using GPT
document = "unpretentious , charming , quirky , original"
chatgpt_generation(prompt, document)

'1'

다음 단계는 전체 테스트 데이터로 오픈AI 모델 중 하나를 실행하는 것입니다. 하지만 전체 테스트 데이터는 1,066개이므로 충분한 크레딧이 있을 때만 실행하세요.

In [ ]:
# 무료 크레딧을 아끼고 싶다면 이 코드를 건너 뛰세요.
predictions = [chatgpt_generation(prompt, doc) for doc in tqdm(data["test"]["text"])]

100%|██████████| 1066/1066 [07:16<00:00,  2.44it/s]


In [ ]:
# 예측을 저장합니다.
y_pred = [int(pred) for pred in predictions]

# 성능을 평가합니다.
evaluate_performance(data["test"]["label"], y_pred)

                 precision    recall  f1-score   support

Negative Review       0.88      0.96      0.92       533
Positive Review       0.96      0.87      0.91       533

       accuracy                           0.92      1066
      macro avg       0.92      0.92      0.92      1066
   weighted avg       0.92      0.92      0.92      1066

